# Regularisierung auf dem Titanic-Datensatz

Hinweis: Da das Notebook nur das Prinizip der Regularisierung zeigen soll, ist der ML-Worflow in diesem Notebook stark vereinfacht (d.h. kein Auffüllen von N/As, kein Feature-Scaling und keine kategorischen Features).

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, RidgeClassifier, RidgeClassifierCV, LassoLars
from sklearn.metrics import accuracy_score

pd.options.mode.chained_assignment = None  # avoid slide-copy-warning 

In [128]:
df = pd.read_csv("data/titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [129]:
df_selection = df[["Survived", "Pclass", "Age", "SibSp", "Parch", "Fare"]]
df_selection = df_selection.dropna()
df_selection.head()

,Survived,Pclass,Age,SibSp,Parch,Fare
0,0,3,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833
2,1,3,26.0,0,0,7.9250
3,1,1,35.0,1,0,53.1000
4,0,3,35.0,0,0,8.0500


In [130]:
df_X = df_selection.drop(columns = ["Survived"])
df_y = df_selection["Survived"]
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=0)

## Aufgaben:

### Aufgabe 1: Ridge-Regularisierung
1. Trainieren Sie eine Logistische Regression mit Ridge-Regularisierung (siehe [RidgeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifier.html)) mit $\alpha =100$ auf den Trainingsdaten.
2. Bestimmen Sie die Accuracy auf den Testdaten.

Aufgabe 1.1

In [131]:
ridgeReg = RidgeClassifier(alpha = 100)
ridgeReg.fit(X_train, y_train)

RidgeClassifier(alpha=100)

Aufgabe 1.2

In [132]:
pred = ridgeReg.predict(X_test)
print("Accuracy auf Testdaten Ridge-Reg:", accuracy_score(y_test, pred))

Accuracy auf Testdaten Ridge-Reg: 0.6713286713286714


### Aufgabe 2: Cross-Validation über $\alpha$
1. Führen Sie nun eine Cross-Validation mit Hilfe der Klasse [RidgeClassifierCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifierCV.html) durch. Benutzen Sie dafür die gleichen Werte für $\alpha$ wie im Notebook [`4_OverUnderfitting.ipynb`](4_OverUnderfitting.ipynb) in Zelle 9.
2. Bestimmen Sie die Accuracy auf den Testdaten.

Aufgabe 2.1

In [133]:
alphas = np.arange(1, 10000, 1)

ridgeClCV = RidgeClassifierCV(alphas = alphas, scoring = "neg_mean_squared_error")
ridgeClCV.fit(X_train,y_train)
ridgeClCV.alpha_

1

Aufgabe 2.2

In [134]:
pred2 = ridgeClCV.predict(X_test)
print("Accuracy auf Testdaten RidgeCV:", accuracy_score(y_test, pred2))

Accuracy auf Testdaten RidgeCV: 0.6853146853146853


### Aufgabe 3:  Lasso-Regularisierung
Für die Lasso-Regularisierung gibt es keine extra Unterklasse, da diese bei der Klasse `LogisitcRegression` über den Parameter `penalty` eingestellt werden kann. Mit dem Parameter `penalty = "l1"` wird die Lasso-Regression verwendet.
Der Regularisierungsfaktor $\alpha$ wird in diesem Fall über den Parameter `C` bestimmt, welcher den Default-Wert `C=1.0` hat:

`LogisticRegression(max_iter=1000, penalty = "l1", C=1.0, solver="liblinear")`

Beachten Sie dabei:
- Für die Regression ist das zusätzliche Argument `solver="liblinear"` nötig, da der Standard-Optimierer nicht mit `L1` funktioniert.
- Parameter `C` gibt die <ins>inverse</ins> Stärke der Regularisierung an (Details siehe [hier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)).


1. Trainieren Sie eine Logistische Regression mit Lasso-Regression für verschiedene Parameter `C`:
    - den Default-Wert `C=1.0`.
    - wählen Sie `C` so, dass das Modell sehr stark regularisiert ist.
    - wählen Sie `C` so, dass das Modell keine Regularisierung hat.
2. Vergleiche Sie die Ergebnisse dieser Modelle indem Sie:
    - die Gewichte und deren Summe ausgeben (wie im Notebook [`4_OverUnderfitting.ipynb`](4_OverUnderfitting.ipynb)).
    - die Accuracy auf den Testdaten ermitteln. 

In [138]:
logR_C1 = LogisticRegression(max_iter=1000, penalty = "l1", C=1.0, solver="liblinear")
logR_C1.fit(X_train, y_train)
pred_normal = logR_C1.predict(X_test)

logR_C_strong = LogisticRegression(max_iter=1000, penalty = "l1", C=0.001, solver="liblinear")
logR_C_strong.fit(X_train, y_train)
pred_strong = logR_C_strong.predict(X_test)

logR_C_weak = LogisticRegression(max_iter=1000, penalty = "l1", C=10000, solver="liblinear")
logR_C_weak.fit(X_train, y_train)
pred_weak = logR_C_weak.predict(X_test)

In [136]:
print("Gewichte der Feature für C=1.0")
# print(logR_C1.coef_[0])
# print(df_X.columns)
print(pd.Series(logR_C1.coef_[0], index = df_X.columns))

print("Gewichte der Feature für C strong")
print(pd.Series(logR_C_strong.coef_[0], index = df_X.columns))

print("Gewichte der Feature für C weak")
print(pd.Series(logR_C_weak.coef_[0], index = df_X.columns))

Gewichte der Feature für C=1.0
Pclass   -0.936959
Age      -0.037661
SibSp    -0.308869
Parch     0.298115
Fare      0.007294
dtype: float64
Gewichte der Feature für C strong
Pclass    0.000000
Age      -0.013057
SibSp     0.000000
Parch     0.000000
Fare      0.006335
dtype: float64
Gewichte der Feature für C weak
Pclass   -1.052788
Age      -0.041773
SibSp    -0.336510
Parch     0.327317
Fare      0.005952
dtype: float64


In [140]:
print("Accuracy für C=1 ->", accuracy_score(y_test,pred_normal))
print("Accuracy für C klein -> stark", accuracy_score(y_test, pred_strong))
print("Accuracy für C groß -> schwach", accuracy_score(y_test, pred_weak))

Accuracy für C=1 -> 0.6923076923076923
Accuracy für C klein -> stark 0.6433566433566433
Accuracy für C groß -> schwach 0.6853146853146853


### Aufgabe 4. Cross-Validierung über  `C`
Ein guter Wert für `C` soll nun mit Hilfe der Methode `cross_val_score` gefunden werden. Ein Beispiel für diese Methode ist:

In [144]:
from sklearn.model_selection import cross_val_score

model = LogisticRegression(max_iter=1000, penalty = "l1", C=1.0, solver="liblinear")
acc_score = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy")
acc_score

array([0.65217391, 0.73684211, 0.6754386 , 0.71929825, 0.71052632])

Die Methode führt eine k-fold Cross-Validation auf den Trainingsdaten durch und gibt für jeden der k-folds den Score (in diesem Fall "accuracy") des Modell auf den jeweiligen Validierungsdaten zurück.

1. Berechnen Sie den Durchschnitt der Rückgabewerte von `cross_val_score` um für das gegebene `model` den durchschnittlichen Accuarcy-Score über alle k-folds zu erhalten.
2. Berechnen Sie für jeden der Werte $ C \in [0.1, 0.2, ...,9.8, 9.9]$ den durchschnittlichen Accuarcy-Score.
3. Trainieren Sie das Modell mit dem besten Wert für $C$ auf allen Trainingsdaten.
4. Bestimmen Sie die Accuracy dieses Modells auf den Testdaten.

Aufgabe 4.1

In [159]:
avg_acc_score = acc_score.mean()
avg_acc_score

arr_score
for c in range(1,99):
    arr_score.append(cross_val_score(model, X_train, y_train, cv=c, scoring="accuracy"))


ValueError: k-fold cross-validation requires at least one train/test split by setting n_splits=2 or more, got n_splits=1.